In [21]:
import numpy as np
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras import optimizers
from sklearn.ensemble import RandomForestClassifier
import os
import glob as gb
import cv2

In [14]:
img_rows, img_cols = 28, 28

In [15]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [16]:
x_train= x_train.astype('float32')
x_test =x_test.astype('float32')
x_train/= 255
x_test /= 255
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(60000, 28, 28) (10000, 28, 28) (60000,) (10000,)


In [17]:
from tensorflow.keras.utils import to_categorical

# Assuming y_train and y_test are already defined
print("Class label of first image:", y_train[0])

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print("After converting the output into a vector:", y_train[0])
print("Shapes:", x_train.shape, x_test.shape, y_train.shape, y_test.shape)

Class label of first image: 5
After converting the output into a vector: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
Shapes: (60000, 28, 28) (10000, 28, 28) (60000, 10) (10000, 10)


In [18]:
#defining these prior to model to increase readability and debugging
batch_size = 128
epochs = 5

# Q) 1.a

# Model-1

In [19]:
def build_model_1():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), strides=2, activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Conv2D(16, (4, 4), strides=2, activation='relu'))
    model.add(MaxPooling2D(pool_size=(4, 4), strides=2))
    model.add(Flatten())
    model.add(Dense(8, activation='relu', name="secondLastLayer"))
    model.add(Dense(10, activation='softmax'))
    model.summary()
    
    adam = optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, batch_size=128, epochs=5, verbose=1, validation_data=(x_test, y_test))

    feature_extractor = Model(inputs=model.inputs, outputs=model.get_layer(name="secondLastLayer").output)
    features_train = feature_extractor.predict(x_train)

    rf_classifier = RandomForestClassifier()
    rf_classifier.fit(features_train, y_train)
    features_test = feature_extractor.predict(x_test)
    accuracy = rf_classifier.score(features_test, y_test)
    print("Random Forest Classifier Accuracy:", accuracy)

    return accuracy

In [20]:
mean = 0
loop_count = 5
for _ in range(loop_count):
    mean += build_model_1()
mean /= loop_count
print(f"Mean for 1.a is {mean}")

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 13, 13, 32)        320       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 12, 12, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 5, 5, 16)          8208      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 1, 1, 16)          0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 16)                0         
                                                                 
 secondLastLayer (Dense)     (None, 8)                

# Q) 1.b

# Model-2

In [22]:
def build_model_2():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), strides=2, activation='relu', input_shape=(28, 28, 1)))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=1))
    model.add(Conv2D(16, (4, 4), strides=2, activation='relu'))
    model.add(AveragePooling2D(pool_size=(4, 4), strides=2))
    model.add(Flatten())
    model.add(Dense(8, activation='relu', name="secondLastLayer"))
    model.add(Dense(10, activation='softmax'))
    model.summary()
    
    adam = optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, batch_size=128, epochs=5, verbose=1, validation_data=(x_test, y_test))

    feature_extractor = Model(inputs=model.inputs, outputs=model.get_layer(name="secondLastLayer").output)
    features_train = feature_extractor.predict(x_train)

    rf_classifier = RandomForestClassifier()
    rf_classifier.fit(features_train, y_train)
    features_test = feature_extractor.predict(x_test)
    accuracy = rf_classifier.score(features_test, y_test)
    print("Random Forest Classifier Accuracy:", accuracy)

    return accuracy

In [23]:
mean = 0
loop_count = 5
for _ in range(loop_count):
    mean += build_model_2()
mean /= loop_count
print(f"Mean for 1.b is {mean}")

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 13, 13, 32)        320       
                                                                 
 average_pooling2d (Average  (None, 12, 12, 32)        0         
 Pooling2D)                                                      
                                                                 
 conv2d_15 (Conv2D)          (None, 5, 5, 16)          8208      
                                                                 
 average_pooling2d_1 (Avera  (None, 1, 1, 16)          0         
 gePooling2D)                                                    
                                                                 
 flatten_7 (Flatten)         (None, 16)                0         
                                                                 
 secondLastLayer (Dense)     (None, 8)                

# Q) 1.c

# Model-3

In [25]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, AveragePooling2D
from tensorflow.keras import optimizers
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import mnist

def build_model_1():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), strides=2, activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Conv2D(16, (4, 4), strides=2, activation='relu'))
    model.add(MaxPooling2D(pool_size=(4, 4), strides=2))
    model.add(Flatten())
    model.add(Dense(8, activation='relu', name="secondLastLayer"))
    model.add(Dense(10, activation='softmax'))
    return model

def build_model_2():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), strides=2, activation='relu', input_shape=(28, 28, 1)))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=1))
    model.add(Conv2D(16, (4, 4), strides=2, activation='relu'))
    model.add(AveragePooling2D(pool_size=(4, 4), strides=2))
    model.add(Flatten())
    model.add(Dense(8, activation='relu', name="secondLastLayer"))
    model.add(Dense(10, activation='softmax'))
    return model

def extract_features_from_model(model, data):
    feature_extractor = Model(inputs=model.inputs, outputs=model.get_layer(name="secondLastLayer").output)
    features = feature_extractor.predict(data)
    return features

def train_random_forest_classifier(features_train, labels_train, features_test, labels_test):
    rf_classifier = RandomForestClassifier()
    rf_classifier.fit(features_train, labels_train)
    predictions = rf_classifier.predict(features_test)
    accuracy = accuracy_score(labels_test, predictions)
    return accuracy

def model_3(x_train, y_train, x_test, y_test):
    model1 = build_model_1()
    model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model1.fit(x_train, y_train, batch_size=128, epochs=5, verbose=1, validation_data=(x_test, y_test))
    
    model2 = build_model_2()
    model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model2.fit(x_train, y_train, batch_size=128, epochs=5, verbose=1, validation_data=(x_test, y_test))
    
    features_train_model1 = extract_features_from_model(model1, x_train)
    features_test_model1 = extract_features_from_model(model1, x_test)
    
    features_train_model2 = extract_features_from_model(model2, x_train)
    features_test_model2 = extract_features_from_model(model2, x_test)
    
    # Stack features horizontally
    stacked_features_train = np.hstack((features_train_model1, features_train_model2))
    stacked_features_test = np.hstack((features_test_model1, features_test_model2))
    
    accuracy = train_random_forest_classifier(stacked_features_train, y_train, stacked_features_test, y_test)
    print("Random Forest Classifier Accuracy:", accuracy)
    
    return accuracy

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]

# Run Model-3
accuracy = model_3(x_train, y_train, x_test, y_test)
print("Accuracy for Model-3:", accuracy)



Epoch 1/5
469/469 [==============================] - 4s 7ms/step - loss: 1.4193 - accuracy: 0.5156 - val_loss: 0.8158 - val_accuracy: 0.7362
Epoch 2/5
469/469 [==============================] - 3s 6ms/step - loss: 0.6939 - accuracy: 0.7811 - val_loss: 0.5760 - val_accuracy: 0.8155
Epoch 3/5
469/469 [==============================] - 3s 7ms/step - loss: 0.5352 - accuracy: 0.8339 - val_loss: 0.4411 - val_accuracy: 0.8639
Epoch 4/5
469/469 [==============================] - 3s 6ms/step - loss: 0.4416 - accuracy: 0.8667 - val_loss: 0.3833 - val_accuracy: 0.8822
Epoch 5/5
469/469 [==============================] - 3s 7ms/step - loss: 0.3796 - accuracy: 0.8861 - val_loss: 0.3280 - val_accuracy: 0.8994
Epoch 1/5
469/469 [==============================] - 4s 7ms/step - loss: 1.6569 - accuracy: 0.4131 - val_loss: 1.0590 - val_accuracy: 0.6487
Epoch 2/5
469/469 [==============================] - 3s 7ms/step - loss: 0.9072 - accuracy: 0.7125 - val_loss: 0.7639 - val_accuracy: 0.7601
Epoch 3/5
46

In [26]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, AveragePooling2D
from tensorflow.keras import optimizers
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import mnist

def build_model_1():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), strides=2, activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Conv2D(16, (4, 4), strides=2, activation='relu'))
    model.add(MaxPooling2D(pool_size=(4, 4), strides=2))
    model.add(Flatten())
    model.add(Dense(8, activation='relu', name="secondLastLayer"))
    model.add(Dense(10, activation='softmax'))
    return model

def build_model_2():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), strides=2, activation='relu', input_shape=(28, 28, 1)))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=1))
    model.add(Conv2D(16, (4, 4), strides=2, activation='relu'))
    model.add(AveragePooling2D(pool_size=(4, 4), strides=2))
    model.add(Flatten())
    model.add(Dense(8, activation='relu', name="secondLastLayer"))
    model.add(Dense(10, activation='softmax'))
    return model

def extract_features_from_model(model, data):
    feature_extractor = Model(inputs=model.inputs, outputs=model.get_layer(name="secondLastLayer").output)
    features = feature_extractor.predict(data)
    return features

def train_random_forest_classifier(features_train, labels_train, features_test, labels_test):
    rf_classifier = RandomForestClassifier()
    rf_classifier.fit(features_train, labels_train)
    predictions = rf_classifier.predict(features_test)
    accuracy = accuracy_score(labels_test, predictions)
    return accuracy

def model_3(x_train, y_train, x_test, y_test):
    model1 = build_model_1()
    model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model1.fit(x_train, y_train, batch_size=128, epochs=5, verbose=1, validation_data=(x_test, y_test))
    
    model2 = build_model_2()
    model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model2.fit(x_train, y_train, batch_size=128, epochs=5, verbose=1, validation_data=(x_test, y_test))
    
    features_train_model1 = extract_features_from_model(model1, x_train)
    features_test_model1 = extract_features_from_model(model1, x_test)
    
    features_train_model2 = extract_features_from_model(model2, x_train)
    features_test_model2 = extract_features_from_model(model2, x_test)
    
    # Stack features horizontally
    stacked_features_train = np.hstack((features_train_model1, features_train_model2))
    stacked_features_test = np.hstack((features_test_model1, features_test_model2))
    
    accuracy = train_random_forest_classifier(stacked_features_train, y_train, stacked_features_test, y_test)
    print("Random Forest Classifier Accuracy:", accuracy)
    
    return accuracy

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]

# Run Model-3 five times
mean_accuracy = 0
num_trials = 5
for i in range(num_trials):
    print(f"Trial {i+1}:")
    accuracy = model_3(x_train, y_train, x_test, y_test)
    mean_accuracy += accuracy
mean_accuracy /= num_trials

print(f"\nMean Accuracy for Model-3 over {num_trials} trials:", mean_accuracy)


Trial 1:
Epoch 1/5
469/469 [==============================] - 4s 7ms/step - loss: 1.2215 - accuracy: 0.5836 - val_loss: 0.6124 - val_accuracy: 0.8007
Epoch 2/5
469/469 [==============================] - 3s 7ms/step - loss: 0.5138 - accuracy: 0.8437 - val_loss: 0.3873 - val_accuracy: 0.8832
Epoch 3/5
469/469 [==============================] - 3s 6ms/step - loss: 0.3812 - accuracy: 0.8857 - val_loss: 0.3081 - val_accuracy: 0.9089
Epoch 4/5
469/469 [==============================] - 3s 6ms/step - loss: 0.3160 - accuracy: 0.9059 - val_loss: 0.2651 - val_accuracy: 0.9200
Epoch 5/5
469/469 [==============================] - 3s 7ms/step - loss: 0.2730 - accuracy: 0.9174 - val_loss: 0.2347 - val_accuracy: 0.9307
Epoch 1/5
469/469 [==============================] - 4s 7ms/step - loss: 1.8984 - accuracy: 0.2913 - val_loss: 1.3881 - val_accuracy: 0.5183
Epoch 2/5
469/469 [==============================] - 3s 6ms/step - loss: 1.1272 - accuracy: 0.6125 - val_loss: 0.9235 - val_accuracy: 0.6879
Epoc

# Q) 1.d

# Model-4

In [30]:
from sklearn.decomposition import PCA
 
def train1():

    # Define Model-1

    model = Sequential()

    model.add(Conv2D(32, (3, 3), strides=2, activation='relu', input_shape=(28, 28, 1)))

    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))

    model.add(Conv2D(16, (4, 4), strides=2, activation='relu'))

    model.add(MaxPooling2D(pool_size=(4, 4), strides=2))

    model.add(Flatten())

    model.add(Dense(8, activation='relu', name="secondLastLayer"))

    model.add(Dense(10, activation='softmax'))

    model.summary()
 
    # Compile and train Model-1

    adam = optimizers.Adam(learning_rate=0.01)

    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(x_train, y_train, batch_size=128, epochs=5, verbose=1, validation_data=(x_test, y_test))
 
    # Extract features from the second last fully connected layer

    feature_extractor = Model(inputs=model.inputs, outputs=model.get_layer(name="secondLastLayer").output)

    features_train = feature_extractor.predict(x_train)
 
    # Train a Random Forest Classifier on the extracted features

    rf_classifier = RandomForestClassifier()

    rf_classifier.fit(features_train, y_train)
 
    # Extract features from the test set and evaluate the classifier

    features_test = feature_extractor.predict(x_test)

    accuracy = rf_classifier.score(features_test, y_test)

    print("Random Forest Classifier Accuracy (Model-1):", accuracy)
 
    return model
 
def train2():

    # Define Model-2

    model = Sequential()

    model.add(Conv2D(32, (3, 3), strides=2, activation='relu', input_shape=(28, 28, 1)))

    model.add(AveragePooling2D(pool_size=(2, 2), strides=1))

    model.add(Conv2D(16, (4, 4), strides=2, activation='relu'))

    model.add(AveragePooling2D(pool_size=(4, 4), strides=2))

    model.add(Flatten())

    model.add(Dense(8, activation='relu', name="secondLastLayer"))

    model.add(Dense(10, activation='softmax'))

    model.summary()
 
    # Compile and train Model-2

    adam = optimizers.Adam(learning_rate=0.01)

    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(x_train, y_train, batch_size=128, epochs=5, verbose=1, validation_data=(x_test, y_test))
 
    # Extract features from the second last fully connected layer

    feature_extractor = Model(inputs=model.inputs, outputs=model.get_layer(name="secondLastLayer").output)

    features_train = feature_extractor.predict(x_train)
 
    # Train a Random Forest Classifier on the extracted features

    rf_classifier = RandomForestClassifier()

    rf_classifier.fit(features_train, np.argmax(y_train, axis=1))
 
    # Extract features from the test set and evaluate the classifier

    features_test = feature_extractor.predict(x_test)

    accuracy = rf_classifier.score(features_test, np.argmax(y_test, axis=1))

    print("Random Forest Classifier Accuracy (Model-2):", accuracy)
 
    return model
 
def train_model_with_pca(model1,model2):

    # Extract features from Model-1

    feature_extractor_model1 = Model(inputs=model1.inputs, outputs=model1.get_layer(name="secondLastLayer").output)

    features_train_model1 = feature_extractor_model1.predict(x_train)
 
    # Extract features from Model-2

    feature_extractor_model2 = Model(inputs=model2.inputs, outputs=model2.get_layer(name="secondLastLayer").output)

    features_train_model2 = feature_extractor_model2.predict(x_train)
 
    # Stack features horizontally

    stacked_features_train = np.hstack((features_train_model1, features_train_model2))
 
    # Apply PCA for dimensionality reduction

    pca = PCA(n_components=8)  # You can experiment with different values of n_components

    reduced_features_train = pca.fit_transform(stacked_features_train)
 
    # Train Random Forest Classifier on reduced features

    rf_classifier = RandomForestClassifier()

    rf_classifier.fit(reduced_features_train, y_train)
 
    # Extract and stack test features, reduce dimensionality with PCA

    features_test_model1 = feature_extractor_model1.predict(x_test)

    features_test_model2 = feature_extractor_model2.predict(x_test)

    stacked_features_test = np.hstack((features_test_model1, features_test_model2))

    reduced_features_test = pca.transform(stacked_features_test)
 
    # Evaluate Random Forest Classifier

    accuracy = rf_classifier.score(reduced_features_test, y_test)

    print("Random Forest Classifier Accuracy (Model-1 and Model-2 with PCA):", accuracy)
 
    return accuracy
 
# Assuming X_train, Y_train, X_test, Y_test are already defined from the data

# Call the functions to train Model-1 and Model-2

model1 = train1()

model2 = train2()
 
# Train Model-3 (stacked features with PCA)

accuracy_model3 = train_model_with_pca(model1,model2)


Model: "sequential_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_82 (Conv2D)          (None, 13, 13, 32)        320       
                                                                 
 max_pooling2d_46 (MaxPooli  (None, 12, 12, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_83 (Conv2D)          (None, 5, 5, 16)          8208      
                                                                 
 max_pooling2d_47 (MaxPooli  (None, 1, 1, 16)          0         
 ng2D)                                                           
                                                                 
 flatten_41 (Flatten)        (None, 16)                0         
                                                                 
 secondLastLayer (Dense)     (None, 8)               

In [33]:
from sklearn.decomposition import PCA
 
def train1():

    # Define Model-1

    model = Sequential()

    model.add(Conv2D(32, (3, 3), strides=2, activation='relu', input_shape=(28, 28, 1)))

    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))

    model.add(Conv2D(16, (4, 4), strides=2, activation='relu'))

    model.add(MaxPooling2D(pool_size=(4, 4), strides=2))

    model.add(Flatten())

    model.add(Dense(8, activation='relu', name="secondLastLayer"))

    model.add(Dense(10, activation='softmax'))

    model.summary()
 
    # Compile and train Model-1

    adam = optimizers.Adam(learning_rate=0.01)

    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(x_train, y_train, batch_size=128, epochs=5, verbose=1, validation_data=(x_test, y_test))
 
    # Extract features from the second last fully connected layer

    feature_extractor = Model(inputs=model.inputs, outputs=model.get_layer(name="secondLastLayer").output)

    features_train = feature_extractor.predict(x_train)
 
    # Train a Random Forest Classifier on the extracted features

    rf_classifier = RandomForestClassifier()

    rf_classifier.fit(features_train, y_train)
 
    # Extract features from the test set and evaluate the classifier

    features_test = feature_extractor.predict(x_test)

    accuracy = rf_classifier.score(features_test, y_test)

    print("Random Forest Classifier Accuracy (Model-1):", accuracy)
 
    return model
 
def train2():

    # Define Model-2

    model = Sequential()

    model.add(Conv2D(32, (3, 3), strides=2, activation='relu', input_shape=(28, 28, 1)))

    model.add(AveragePooling2D(pool_size=(2, 2), strides=1))

    model.add(Conv2D(16, (4, 4), strides=2, activation='relu'))

    model.add(AveragePooling2D(pool_size=(4, 4), strides=2))

    model.add(Flatten())

    model.add(Dense(8, activation='relu', name="secondLastLayer"))

    model.add(Dense(10, activation='softmax'))

    model.summary()
 
    # Compile and train Model-2

    adam = optimizers.Adam(learning_rate=0.01)

    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(x_train, y_train, batch_size=128, epochs=5, verbose=1, validation_data=(x_test, y_test))
 
    # Extract features from the second last fully connected layer

    feature_extractor = Model(inputs=model.inputs, outputs=model.get_layer(name="secondLastLayer").output)

    features_train = feature_extractor.predict(x_train)
 
    # Train a Random Forest Classifier on the extracted features

    rf_classifier = RandomForestClassifier()

    rf_classifier.fit(features_train, np.argmax(y_train, axis=1))
 
    # Extract features from the test set and evaluate the classifier

    features_test = feature_extractor.predict(x_test)

    accuracy = rf_classifier.score(features_test, np.argmax(y_test, axis=1))

    print("Random Forest Classifier Accuracy (Model-2):", accuracy)
 
    return model
 
def train_model_with_pca(model1,model2):

    # Extract features from Model-1

    feature_extractor_model1 = Model(inputs=model1.inputs, outputs=model1.get_layer(name="secondLastLayer").output)

    features_train_model1 = feature_extractor_model1.predict(x_train)
 
    # Extract features from Model-2

    feature_extractor_model2 = Model(inputs=model2.inputs, outputs=model2.get_layer(name="secondLastLayer").output)

    features_train_model2 = feature_extractor_model2.predict(x_train)
 
    # Stack features horizontally

    stacked_features_train = np.hstack((features_train_model1, features_train_model2))
 
    # Apply PCA for dimensionality reduction

    pca = PCA(n_components=8)  # You can experiment with different values of n_components

    reduced_features_train = pca.fit_transform(stacked_features_train)
 
    # Train Random Forest Classifier on reduced features

    rf_classifier = RandomForestClassifier()

    rf_classifier.fit(reduced_features_train, y_train)
 
    # Extract and stack test features, reduce dimensionality with PCA

    features_test_model1 = feature_extractor_model1.predict(x_test)

    features_test_model2 = feature_extractor_model2.predict(x_test)

    stacked_features_test = np.hstack((features_test_model1, features_test_model2))

    reduced_features_test = pca.transform(stacked_features_test)
 
    # Evaluate Random Forest Classifier

    accuracy = rf_classifier.score(reduced_features_test, y_test)

    print("Random Forest Classifier Accuracy (Model-1 and Model-2 with PCA):", accuracy)
 
    return accuracy
 
# Assuming X_train, Y_train, X_test, Y_test are already defined from the data

# Call the functions to train Model-1 and Model-2

model1 = train1()

model2 = train2()
 
# Train Model-3 (stacked features with PCA)

# accuracy_model3 = train_model_with_pca(model1,model2)

# Run Model-4 five times
mean_accuracy = 0
num_trials = 5
for i in range(num_trials):
    print(f"Trial {i+1}:")
    mean_accuracy += train_model_with_pca(model1,model2)
mean_accuracy /= num_trials

print(f"\nMean Accuracy for Model-4 over {num_trials} trials:", mean_accuracy)



Model: "sequential_53"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_106 (Conv2D)         (None, 13, 13, 32)        320       
                                                                 
 max_pooling2d_58 (MaxPooli  (None, 12, 12, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_107 (Conv2D)         (None, 5, 5, 16)          8208      
                                                                 
 max_pooling2d_59 (MaxPooli  (None, 1, 1, 16)          0         
 ng2D)                                                           
                                                                 
 flatten_53 (Flatten)        (None, 16)                0         
                                                                 
 secondLastLayer (Dense)     (None, 8)               

# Q) 1.e

### Model-4 is the best model among the above four models for classifying MNIST dataset because of the dimensionality reduction and reduced model complexity after being horizontally staked, due to improvement in performance evaluation.